In [1]:
class SINDy(torch.Tensor):
    """
    SINDy dz predictions

    Arguments:
        params - Dictionary object containing the parameters that specify the training.
        See params.txt file for a description of the parameters.

    Returns:
        sindy_predict - tensor containing sindy's predictions for dz.
    """


    def __init__(self, params:Dict = {}, encoder:AutoEncoder = torch.Tensor, *args, **kwargs) -> None:
      super().__init__(*args, **kwargs) #what for?
      self.params = params
      self.encoder = encoder

      self.input_dim = self.params['input_dim']
      self.latent_dim = self.params['latent_dim']
      self.poly_order = self.params['poly_order']
      if 'include_sine' in self.params.keys():
        self.include_sine = self.params['include_sine']
      else:
        self.include_sine = False
      self.library_dim = self.params['library_dim']
      self.model_order = self.params['model_order']
      self.sequential_thresholding = self.params['sequential_thresholding']
      self_coefficient_initialization = self.params['coefficient_initialization']
      'self_coefficient_mask = self.params['coefficient_initialization']

      self.sindy_coefficients = torch.Tensor(self.library_dim, self.latent_dim)


    def init_sindy_coefficients(self, name='normal', std=1.):

      self.sindy_coefficients = std*torch.randn_like(self.sindy_coefficients)

      ''' 
      if name == 'xavier':
        self.sindy_coefficients = torch.nn.init.xavier_uniform_(self.sindy_coefficients)
      elif name == 'uniform':
        self.sindy_coefficients = torch.nn.init.uniform_(self.sindy_coefficients, low=0.0, high=1.0)
      elif name == 'normal':
        self.sindy_coefficients = torch.nn.init.normal_(self.sindy_coefficients, mean=0, std=1) 
      '''

    def predict(self)-> torch.Tensor:

      #create Theta
      if self.model_order == 1:
        Theta = sindy_library_tf(self.encoder.z, self.latent_dim, self.poly_order, self.include_sine)
      else:
        Theta = sindy_library_tf_order2(self.encoder.z, self.encoder.dz, self.latent_dim, self.poly_order, self.include_sine)
      
      #initialize sindy coefficients
      init_sindy_coefficients()

      #apply thresholding or not
      if self.sequential_thresholding:
        '''
        tmp = torch.rand(size=(library_dim,latent_dim), dtype=torch.float32)
        mask = torch.zeros_like(tmp)
        mask = mask.where(self.coefficient_mask, tmp)
        '''
        mask = torch.rand(size=(library_dim,latent_dim), dtype=torch.float32)
        sindy_predict = torch.matmul(Theta, mask*sindy_coefficients)
      else:
        sindy_predict = tf.matmul(Theta, sindy_coefficients)

    return sindy_predict

SyntaxError: unterminated string literal (detected at line 30) (3315716334.py, line 30)